In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
from colorama import Fore, Style
from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import randint
from lightgbm import LGBMRegressor
import lightgbm as lgb
from math import pi
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

: 

In [ ]:
df_tr = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
df_ts = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
df_s = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv')



In [ ]:
print(df_ts.columns)

In [ ]:
print(df_tr.columns)

In [ ]:
columns_to_drop = ['shutdown', 'mini_shutdown','blackout', 'mov_change', 'frankfurt_shutdown',
       'precipitation', 'snow', 'user_activity_1', 'user_activity_2',]
df_tr = df_tr.drop(columns_to_drop, axis=1)

In [ ]:
df_tr.describe(include="all")

In [ ]:
df_ts.describe(include="all")

In [ ]:
print(df_tr.info())
print("--------------------------------------------------")
print(df_ts.info())

In [ ]:
df_tr.drop(['id'], axis=1 , inplace=True)
df_ts.drop(['id'], axis=1, inplace=True)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.ensemble import VotingRegressor

# Load datasets
train = df_tr.copy()
test = df_ts.copy()

# Feature Engineering
def feature_engineering(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)
    df['quarter'] = df['date'].dt.quarter
    
    le = LabelEncoder()
    df['warehouse'] = le.fit_transform(df['warehouse'])
    df['holiday_name'] = df['holiday_name'].fillna('No Holiday')
    df['holiday_name'] = le.fit_transform(df['holiday_name'])
    
    df = df.drop(['date'], axis=1)
    
    return df

train = feature_engineering(train)
test = feature_engineering(test)

# Split data
X = train.drop(['orders'], axis=1)
y = train['orders']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1: LightGBM
lgb_model = LGBMRegressor()
lgb_model.fit(X_train, y_train)
lgb_val_preds = lgb_model.predict(X_val)
lgb_mape = mean_absolute_percentage_error(y_val, lgb_val_preds)
print(f'LightGBM MAPE: {lgb_mape}')

# Model 2: XGBoost
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_val_preds = xgb_model.predict(X_val)
xgb_mape = mean_absolute_percentage_error(y_val, xgb_val_preds)
print(f'XGBoost MAPE: {xgb_mape}')

# Model 3: Neural Network (LSTM)
X_train_nn = np.expand_dims(X_train, axis=1)
X_val_nn = np.expand_dims(X_val, axis=1)

nn_model = Sequential()
nn_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_nn.shape[1], X_train_nn.shape[2])))
nn_model.add(Dropout(0.2))
nn_model.add(LSTM(units=50, return_sequences=False))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(units=1))

nn_model.compile(optimizer='adam', loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
nn_model.fit(X_train_nn, y_train, epochs=50, batch_size=32, validation_data=(X_val_nn, y_val), callbacks=[early_stopping])

nn_val_preds = nn_model.predict(X_val_nn).flatten()
nn_mape = mean_absolute_percentage_error(y_val, nn_val_preds)
print(f'LSTM Neural Network MAPE: {nn_mape}')

# Ensemble Model
ensemble_model = VotingRegressor(estimators=[
    ('lgb', lgb_model),
    ('xgb', xgb_model),
    ('nn', keras.wrappers.scikit_learn.KerasRegressor(build_fn=lambda: nn_model, epochs=1, batch_size=32, verbose=0))
])

ensemble_model.fit(X_train, y_train)
ensemble_val_preds = ensemble_model.predict(X_val)
ensemble_mape = mean_absolute_percentage_error(y_val, ensemble_val_preds)
print(f'Ensemble Model MAPE: {ensemble_mape}')

# Make predictions on the test set
test_preds = ensemble_model.predict(test)

# Prepare submission
submission = pd.DataFrame({'date': pd.read_csv('test.csv')['date'], 'orders': test_preds})
submission.to_csv('submission.csv', index=False)


# Something

In [ ]:
df_tr.drop(['holiday_name', 'holiday', 'shops_closed','winter_school_holidays', 'school_holidays'], axis=1 , inplace=True)
df_ts.drop(['holiday_name', 'holiday', 'shops_closed','winter_school_holidays', 'school_holidays'], axis=1, inplace=True)

In [ ]:
print(df_tr.info())
print("--------------------------------------")
print(df_ts.info())

In [ ]:
# Function to encode date cyclically with advanced features
def advanced_cyclic_date_encoding(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col], format='%Y-%m-%d')
    # Extract day, month, year, day of year, weekday, week of year, and quarter
    df['day'] = df[date_col].dt.day
    df['month'] = df[date_col].dt.month
    df['year'] = df[date_col].dt.year
    df['day_of_year'] = df[date_col].dt.dayofyear
    df['weekday'] = df[date_col].dt.weekday
    df['week_of_year'] = df[date_col].dt.isocalendar().week
    df['quarter'] = df[date_col].dt.quarter
    
    # Normalize these features
    df['day_norm'] = df['day'] / 31.0
    df['month_norm'] = df['month'] / 12.0
    df['day_of_year_norm'] = df['day_of_year'] / 366.0
    df['weekday_norm'] = df['weekday'] / 7.0
    df['week_of_year_norm'] = df['week_of_year'] / 53.0
    df['quarter_norm'] = df['quarter'] / 4.0
    
    # Apply sine and cosine transformations
    df['day_sin'] = np.sin(2 * np.pi * df['day_norm'])
    df['day_cos'] = np.cos(2 * np.pi * df['day_norm'])
    df['month_sin'] = np.sin(2 * np.pi * df['month_norm'])
    df['month_cos'] = np.cos(2 * np.pi * df['month_norm'])
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year_norm'])
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year_norm'])
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday_norm'])
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday_norm'])
    df['week_of_year_sin'] = np.sin(2 * np.pi * df['week_of_year_norm'])
    df['week_of_year_cos'] = np.cos(2 * np.pi * df['week_of_year_norm'])
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter_norm'])
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter_norm'])
    
    return df

In [ ]:
df_tr = advanced_cyclic_date_encoding(df_tr, 'date')
df_tr

In [ ]:
df_tr.drop(['id', 'date'], axis=1,inplace=True)

In [ ]:
le_warehouse = LabelEncoder()
df_tr['warehouse'] = le_warehouse.fit_transform(df_tr['warehouse'])
df_ts['warehouse'] = le_warehouse.transform(df_ts['warehouse'])

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Calculate Correlation Matrix
corr_matrix = df_tr.corr()

# Plot using Seaborn Heatmap
plt.figure(figsize=(18, 12))  # Adjust figure size as needed
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
# Calculate Correlation Matrix
corr_matrix = df_tr.corr()

# Threshold for High Correlation
threshold = 0.75

# Filter Features to Drop (Highly Correlated with Others)
features_to_drop = set()
for col in corr_matrix.columns:
    for other_col in corr_matrix.columns:
        if col != other_col:  # Avoid self-correlation
            corr = abs(corr_matrix.loc[col, other_col])
            if corr >= threshold:
                features_to_drop.add(max(col, other_col))  # Drop the higher-indexed feature

# Drop Featuresa
df_tr = df_tr.drop(features_to_drop, axis=1)
df_tr.columns

In [ ]:
X = df_tr.drop('orders', axis=1)
y = df_tr['orders']

In [ ]:
df_ts

In [ ]:
df_ts = advanced_cyclic_date_encoding(df_ts, 'date')
df_ts

In [ ]:
df_ts.drop(['id', 'date'], axis=1, inplace=True)

In [ ]:
df_test=df_ts[['warehouse', 'day', 'year', 'day_of_year', 'weekday',
       'day_cos', 'day_of_year_cos', 'weekday_sin', 'weekday_cos']].copy()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
import numpy as np
import pandas as pd

# Define the models
models = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42)),
    ('lgbm', LGBMRegressor(objective='regression', random_state=42)),
    ('catboost', CatBoostRegressor(iterations=200,random_state=42)),
]

# Define the KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the MAPE scores
mape_scores = np.zeros((len(models),))

# Loop through each model
for i, (name, model) in enumerate(models):
    mape_scores_model = []
    # Loop through each fold
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the validation set
        y_pred = model.predict(X_val)
        
        # Calculate MAPE
        mape = mean_absolute_percentage_error(y_val, y_pred)
        mape_scores_model.append(mape)
    
    # Calculate the average MAPE score for this model
    mape_scores[i] = np.mean(mape_scores_model)

In [ ]:
# Print the MAPE scores for each model
for i, (name, model) in enumerate(models):
    print(f"Model: {name}, MAPE: {mape_scores[i]:.5f}%")

In [ ]:
# Create the ensemble model with soft voting
weights = 1 / mape_scores
ensemble = VotingRegressor(estimators=models, weights=weights)

# Evaluate the ensemble model with 5-fold cross-validation
mape_scores_ensemble = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    ensemble.fit(X_train, y_train)
    y_pred = ensemble.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    mape_scores_ensemble.append(mape)

In [ ]:
mape_ensemble = np.mean(mape_scores_ensemble)
print(f"Ensemble MAPE: {mape_ensemble:.5f}%")

In [ ]:
# Determine the best model
best_model_index = np.argmin(mape_scores)
best_model = models[best_model_index][1]
best_mape = mape_scores[best_model_index]

if mape_ensemble < best_mape:
    print("Ensemble model performs better.")
    final_model = ensemble
else:
    print(f"{models[best_model_index][0]} model performs better.")
    final_model = best_model

# Generate predictions on the test set using the best model
final_model.fit(X, y)
y_pred_test = final_model.predict(df_test)

# You can now use y_pred_test for further analysis or to create a submission file

In [ ]:
y_pred_test

In [ ]:
df_s['orders']=y_pred_test
df_s

In [ ]:
df_s = df_s.set_index('id')
df_s.to_csv("submission.csv")

_________
_______
_____

In [ ]:
df_tr

In [ ]:
df_ts

In [ ]:
full=pd.concat([df_tr, df_ts], axis=0)

In [ ]:
full

In [ ]:
full.reset_index(inplace=True)

In [ ]:
df = advanced_cyclic_encoding(full, 'date', fourier_components=7737)
df

In [ ]:
def split_last_397(df):
    test_df = df.iloc[-397:]
    train_df = df.iloc[:-397]
    return train_df, test_df

train_df, test_df = split_last_397(df.copy())  # Copy to avoid modifying original df


In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
train_df.drop(['index','id'], axis=1, inplace=True)
test_df.drop(['index','id', 'orders'], axis=1, inplace=True)

In [ ]:
le_warehouse = LabelEncoder()
train_df['warehouse'] = le_warehouse.fit_transform(train_df['warehouse'])
test_df['warehouse'] = le_warehouse.transform(test_df['warehouse'])

In [ ]:
min_val = train_df['fourier_year'].min()
max_val = train_df['fourier_year'].max()

train_df['fourier_year'] = (train_df['fourier_year'] - min_val) / (max_val - min_val)

In [ ]:
min_val = test_df['fourier_year'].min()
max_val = test_df['fourier_year'].max()

test_df['fourier_year'] = (test_df['fourier_year'] - min_val) / (max_val - min_val)

In [ ]:
X = train_df.drop('orders', axis=1)
y = train_df['orders']

In [ ]:
# Define the models
models = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror', random_state=42)),
    ('lgbm', LGBMRegressor(objective='regression', random_state=42)),
    ('catboost', CatBoostRegressor(iterations=200,random_state=42)),
]

# Define the KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the MAPE scores
mape_scores = np.zeros((len(models),))

# Loop through each model
for i, (name, model) in enumerate(models):
    mape_scores_model = []
    # Loop through each fold
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the validation set
        y_pred = model.predict(X_val)
        
        # Calculate MAPE
        mape = mean_absolute_percentage_error(y_val, y_pred)
        mape_scores_model.append(mape)
    
    # Calculate the average MAPE score for this model
    mape_scores[i] = np.mean(mape_scores_model)

In [ ]:
# Print the MAPE scores for each model
for i, (name, model) in enumerate(models):
    print(f"Model: {name}, MAPE: {mape_scores[i]:.5f}%")

In [ ]:
# Create the ensemble model with soft voting
weights = 1 / mape_scores
ensemble = VotingRegressor(estimators=models, weights=weights)

# Evaluate the ensemble model with 5-fold cross-validation
mape_scores_ensemble = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    ensemble.fit(X_train, y_train)
    y_pred = ensemble.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    mape_scores_ensemble.append(mape)

In [ ]:
mape_ensemble = np.mean(mape_scores_ensemble)
print(f"Ensemble MAPE: {mape_ensemble:.5f}%")

In [ ]:
# Determine the best model
best_model_index = np.argmin(mape_scores)
best_model = models[best_model_index][1]
best_mape = mape_scores[best_model_index]

if mape_ensemble < best_mape:
    print("Ensemble model performs better.")
    final_model = ensemble
else:
    print(f"{models[best_model_index][0]} model performs better.")
    final_model = best_model

# Generate predictions on the test set using the best model
final_model.fit(X, y)
y_pred_test = final_model.predict(test_df)

y_pred_test

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.api as sm
from scipy import fftpack
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error

def fuck_df(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    # ---------- Basic Date Components ----------
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_month'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  
    df['quarter'] = df['date'].dt.quarter


    # ---------- Cyclical Encoding ----------
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
    df['week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    # ---------- Trend and Seasonality Decomposition ----------
    stl = STL(df['orders'], period=365)  # Assuming yearly seasonality
    res = stl.fit()
    df['trend'] = res.trend
    df['seasonality'] = res.seasonal

    # ---------- Autocorrelation Features ----------
    lags = [7, 14, 28, 56]  # Example lags to consider
    for lag in lags:
        df[f'autocorr_{lag}'] = acf(df['orders'], nlags=lag)[-1]
        df[f'pacf_{lag}'] = pacf(df['orders'], nlags=lag)[-1]


    # ---------- Time-Varying Features (Exponential Smoothing) ----------
    # Simple Exponential Smoothing
    alpha = 0.2  # You can optimize this value
    df['ses'] = df['orders'].ewm(alpha=alpha, adjust=False).mean()

    # Double Exponential Smoothing
    df['des_level'] = df['orders'].ewm(alpha=alpha, adjust=False).mean()
    df['des_trend'] = (df['des_level'] - df['des_level'].shift(1)).ewm(alpha=alpha, adjust=False).mean()

    # ---------- Frequency Domain Features (FFT) ----------
    fft = fftpack.fft(df['orders'].values)  # Convert Series to numpy array.
    df['fft_amplitude'] = np.abs(fft)
    df['fft_phase'] = np.angle(fft)
    
    # ---------- Interaction Terms (Example) ----------
    df['month_is_weekend'] = df['month'] * df['is_weekend']
    
    return df

In [ ]:
df_tr=fuck_df(df_tr.copy())

In [ ]:
df_tr

In [ ]:
# Define target variable and features
y = df['orders'] 
X = df.drop('orders', axis=1)

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
mape_scores = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train XGBoost model
    model = XGBRegressor()  
    model.fit(X_train, y_train)

    # Make Predictions
    y_pred = model.predict(X_test)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_test, y_pred)
    mape_scores.append(mape)

# Average MAPE across folds
avg_mape = sum(mape_scores) / len(mape_scores)
print("Average MAPE:", avg_mape)

In [ ]:
df_tr['holiday_name'].fillna('noholiday', inplace=True)
df_ts['holiday_name'].fillna('noholiday', inplace=True)

In [ ]:
df_tr = df_tr.drop(['id'], axis=1)
df_ts = df_ts.drop(['id'], axis=1)

In [ ]:
object_cols = df_tr.select_dtypes(include='object').columns

# Print unique values for each object column
for col in object_cols:
  unique_values = df_tr[col].unique()
  print(f"Unique values in column '{col}': {unique_values}")

In [ ]:
object_cols = df_ts.select_dtypes(include='object').columns

# Print unique values for each object column
for col in object_cols:
  unique_values = df_ts[col].unique()
  print(f"Unique values in column '{col}': {unique_values}")

In [ ]:
le_warehouse = LabelEncoder()
le_holiday_name = LabelEncoder()

# Fit and transform training data

df_tr['warehouse'] = le_warehouse.fit_transform(df_tr['warehouse'])
df_tr['holiday_name'] = le_holiday_name.fit_transform(df_tr['holiday_name'])

# Transform test data using the fitted encoders from training
df_ts['warehouse'] = le_warehouse.transform(df_ts['warehouse'])
df_ts['holiday_name'] = le_holiday_name.transform(df_ts['holiday_name'])

In [ ]:
def cyclic_encode_date(df, date_col):

  # Extract year, month, day (assuming the format YYYY-MM-DD)
    df['year'] = pd.to_datetime(df[date_col]).dt.year - 2020
    df['month'] = pd.to_datetime(df[date_col]).dt.month / 12  # Normalize to 0-1 range
    df['day'] = pd.to_datetime(df[date_col]).dt.day / 31  # Normalize to 0-1 range (assuming 31 days)

      # Apply sine and cosine transformations
    df['year_sin'] = df['year'] * np.sin(2 * pi * df['year'])
    df['year_cos'] = df['year'] * np.cos(2 * pi * df['year'])
    df['month_sin'] = df['month'] * np.sin(2 * pi * df['month'])
    df['month_cos'] = df['month'] * np.cos(2 * pi * df['month'])
    df['day_sin'] = df['day'] * np.sin(2 * pi * df['day'])
    df['day_cos'] = df['day'] * np.cos(2 * pi * df['day'])
    return df

# Apply cyclic encoding to your DataFrame
df_tr = cyclic_encode_date(df_tr,'date')
df_ts = cyclic_encode_date(df_ts,'date')

In [ ]:
df_tr.drop('date', axis=1,inplace=True)
df_ts.drop('date', axis=1,inplace=True)

In [ ]:
df_tr

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Calculate Correlation Matrix
corr_matrix = df_tr.corr()

# Plot using Seaborn Heatmap
plt.figure(figsize=(18, 12))  # Adjust figure size as needed
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
# Calculate Correlation Matrix
corr_matrix = df_tr.corr()

# Threshold for High Correlation
threshold = 0.75

# Filter Features to Drop (Highly Correlated with Others)
features_to_drop = set()
for col in corr_matrix.columns:
    for other_col in corr_matrix.columns:
        if col != other_col:  # Avoid self-correlation
            corr = abs(corr_matrix.loc[col, other_col])
            if corr >= threshold:
                features_to_drop.add(max(col, other_col))  # Drop the higher-indexed feature

# Drop Featuresa
df_tr = df_tr.drop(features_to_drop, axis=1)

In [ ]:
df_tr.columns

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Calculate Correlation Matrix
corr_matrix = df_ts.corr()

# Plot using Seaborn Heatmap
plt.figure(figsize=(18, 12))  # Adjust figure size as needed
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
df_ts=df_ts[['warehouse', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'year', 'month', 'day',
       'month_sin', 'month_cos', 'day_cos']].copy()

In [ ]:
df_ts.columns

In [ ]:
print(df_tr.info())
print(df_ts.info())

In [ ]:
X = df_tr.drop('orders', axis=1)
y = df_tr['orders']

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_cols = X.select_dtypes(include=['float64', 'int64', 'int32']).columns
scaler = StandardScaler()
scaler.fit(X[numeric_cols])  # Fit the scaler to your training data (important!)

# Transform the data
X_scaled = X.copy()
X_scaled[numeric_cols] = scaler.transform(X[numeric_cols])


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_val, y_pred)

print("Mean Absolute Percentage Error (MAPE):", mape)

In [ ]:
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

# Define the objective function
def objective(params):
    # Define the XGBoost model with the given parameters
    model = xgb.XGBRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda']
    )

    # Calculate the MAPE using cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_percentage_error')
    # Negate the score as hyperopt minimizes the objective function
    mape = -cv_scores.mean()

    return mape

# Define the search space for hyperparameters
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 50),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'gamma': hp.loguniform('gamma', -5, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, 0),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 0)
}

# Create a trials object to store the results
trials = Trials()

# Run the optimization
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,  # Number of trials
    trials=trials
)

# Print the best parameters found
print("Best parameters:", best_params)

# Train the XGBoost model with the best parameters
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the MAPE on the test set
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE on test set:", mape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Custom MAPE Loss Function
def mape_loss(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()  
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    diff = tf.abs((y_true - y_pred) / tf.keras.backend.maximum(tf.abs(y_true), epsilon))
    return 100. * tf.keras.backend.mean(diff)

# Advanced Dense Layer Modela
model = keras.Sequential([
    layers.Input(shape=(15,)),  # Assuming 15 features
    layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(1, activation='linear') 
])

# Compile Model
model.compile(loss=mape_loss, optimizer='adam', metrics=['mean_absolute_percentage_error'])

# Fit Model
history = model.fit(X_train, y_train, epochs=200, validation_split=0.15)

# Plot Training MAPE Loss
plt.plot(history.history['mean_absolute_percentage_error'])
plt.title('Training MAPE Loss')
plt.xlabel('Epoch')
plt.ylabel('MAPE (%)')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Custom MAPE Loss Function
def mape_loss(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()  
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    diff = tf.abs((y_true - y_pred) / tf.keras.backend.maximum(tf.abs(y_true), epsilon))
    return 100. * tf.keras.backend.mean(diff)

# Advanced Dense Layer Model
model = keras.Sequential([
    layers.Input(shape=(15,)),  # Assuming 15 features
    layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(1, activation='linear') 
])

# Compile Model
model.compile(loss=mape_loss, optimizer='adam', metrics=['mean_absolute_percentage_error'])

# Define early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

# Define learning rate scheduler
lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * 0.9 ** (epoch // 10))

# Fit Model
history = model.fit(X_train, y_train, epochs=200, validation_split=0.15, 
                    callbacks=[early_stopping, lr_scheduler])

# Plot Training MAPE Loss
plt.plot(history.history['mean_absolute_percentage_error'])
plt.title('Training MAPE Loss')
plt.xlabel('Epoch')
plt.ylabel('MAPE (%)')
plt.show()

In [ ]:
%%capture
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon

In [ ]:
import pandas as pd

In [ ]:
df_tr = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
df_ts = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
df_s = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv')

In [ ]:
columns_to_drop = ['shutdown', 'mini_shutdown','blackout', 'mov_change', 'frankfurt_shutdown',
       'precipitation', 'snow', 'user_activity_1', 'user_activity_2',]
df_tr = df_tr.drop(columns_to_drop, axis=1)

In [ ]:
df_tr['holiday_name'].fillna('noholiday', inplace=True)
df_ts['holiday_name'].fillna('noholiday', inplace=True)

In [ ]:
df_tr = df_tr.drop(['id'], axis=1)
df_ts = df_ts.drop(['id'], axis=1)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
predictor = TabularPredictor(label='orders',path="autogluon-daily",eval_metric="mape",).fit(df_tr, presets='best_quality', time_limit=3600)

In [ ]:
predictor.leaderboard()

In [ ]:
preds=predictor.predict(df_ts)

In [ ]:
preds=predictor.predict(df_ts, model='NeuralNetFastAI_BAG_L2')

In [ ]:
preds

In [ ]:
df_s = pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv')
df_s['orders']=preds


In [ ]:
df_s

In [ ]:
df_s = df_s.set_index('id')
df_s.to_csv("submission.csv")